In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint

from keras.preprocessing import image

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,GlobalAveragePooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import warnings
warnings.filterwarnings('ignore')

In [3]:
model_vgg16 = VGG16(weights='imagenet', include_top=False)

In [5]:
for layer in model_vgg16.layers:
    layer.trainable = False

In [6]:
img_width, img_height = 150, 150
train_data_dir = '/content/drive/MyDrive/train_new'
val_data_dir = '/content/drive/MyDrive/test_new'
model_weights_file = 'model_vgg16.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

In [7]:
put1 = Input(shape=(img_width, img_height, 3))
output_resnet = model_vgg16(put1)
x = Flatten()(output_resnet)
x = Dense(64, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
model=Model(inputs=put1,outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=32,class_mode='categorical')

Found 7414 images belonging to 10 classes.
Found 1847 images belonging to 10 classes.


In [9]:
# callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, epochs=1,
  steps_per_epoch=len(train_generator),
  validation_steps=len(validation_generator))

Epoch 1/4
232/232 [==============================] - 1635s 7s/step - loss: 2.1751 - accuracy: 0.3552
Epoch 2/4
232/232 [==============================] - 1618s 7s/step - loss: 0.9909 - accuracy: 0.6703
Epoch 3/4
232/232 [==============================] - 1617s 7s/step - loss: 0.6602 - accuracy: 0.7823
Epoch 4/4
232/232 [==============================] - 1639s 7s/step - loss: 0.5008 - accuracy: 0.8455


In [13]:
import os

In [14]:
class_names=[a for a in os.listdir('/content/drive/MyDrive/train_new')]
print(class_names)

['doberman', 'malamute', 'beagle', 'tibetan_mastiff', 'pug', 'scottish_deerhound', 'golden_retriever', 'saint_bernard', 'chihuahua', 'french_bulldog']


In [10]:
pip install gradio

     |████████████████████████████████| 1.5MB 4.4MB/s 
     |████████████████████████████████| 215kB 41.4MB/s 
     |████████████████████████████████| 1.9MB 24.4MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 962kB 36.5MB/s 
     |████████████████████████████████| 3.2MB 29.1MB/s 
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp37-none-any.whl size=3372 sha256=ec91ada0a7b7be38d2728817df4354fbcc1094f45d4e84edd1a596cf685dc41e
  Stored in directory: /root/.cache/pip/wheels/9f/fc/a7/ab5712c3ace9a8f97276465cc2937316ab8063c1fea488ea77
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp37-none-any.whl size=4710 sha256=13332c8f99c61b0d7cc9ae685cbe971ae3ef52204471a8eb74696a767ed39984
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
Successfully built flask-cachebuster ffmpy


In [11]:
import gradio as gr

In [15]:
def predict_image(img):
  img_4d=img.reshape(-1,150,150,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(10)}

In [25]:
image = gr.inputs.Image(shape=(150,150))
label = gr.outputs.Label(num_top_classes=10)

rq=gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')
rq.launch()